In [166]:
from numpy import random, exp, array, dot 
import pandas as pd
from matplotlib import pyplot as plt 
%matplotlib inline
import skimage.io as io

In [177]:
class NeuralNet():
    def __init__(self,x,y):
        
        # Seed the random generator 
        random.seed(1) 
        # random.seed fixes that random values so that answer is same
        
        # Save all variables in self for future references
        self.x = x 
        self.y = y
        self.x_shape = 784
        self.y_shape = 10
        self.layer_1_nodes = 4
        self.layer_2_nodes = 5
        self.layer_3_nodes = 3
        
        # generate weights with value -1 to 1
        self.weights_1 = 2 * random.random((self.x_shape ,self.layer_1_nodes)) - 1
        self.weights_2 = 2 * random.random((self.layer_1_nodes,self.layer_2_nodes)) - 1
        self.weights_3 = 2 * random.random((self.layer_2_nodes, self.layer_3_nodes)) - 1
        self.out_weights = 2 * random.random((self.layer_3_nodes, self.y_shape)) - 1
        
    # Sigmoid 
    def sigmoid(self,x):
        return 1 / (1 + exp(x))
        
    # Derivative sigmoid 
    def sigmoid_derivative(self,x):
        return x * (1-x)
        
    def softmax(self,x):
        return exp(x) / sum(exp(x))
    
    def softmax_derivative(self,x):
        #https://deepnotes.io/softmax-crossentropy
        pj = softmax = exp(x) / sum(exp(x))
        
        #derivative = del-pj / del-ai 
        derivative = pi * (1-pj) # if i = j 
        derivative = -pi * pj # if i != j
        # How to write this code??? what are i and j 
            
    def predict(self,x):
        # Multiply the input weights and find it's sigmoid 
        print x.shape
        print self.weights_1.shape
        layer_1 = self.sigmoid(dot(x,self.weights_1)) #1 
        layer_2 = self.sigmoid(dot(layer_1,self.weights_2)) #10 
        layer_3 = self.sigmoid(dot(layer_2,self.weights_3)) #28
        output = self.softmax(dot(layer_3,self.out_weights)) #28x10
        print output.shape
        return output
    
    # X -->(w1)  Z1 -->Sigmoid  A1 -->(w2) --> Z2 -->(softmax) Y
    
    # del(L) / del(w2)
    # Z2 = W2.T * A1
    # Z1 = W1.T * x
    # A1 = sigmoid(z1)
    # Y = sigmoid(Z2)
    
    
    def train(self,num_steps):
        errors = []
        
        # testing ka hain ye 
        # train karo naa ki predict
        
        for x in range(num_steps):
            print x
            
            # FORWARD 
            layer_1 = self.sigmoid(dot(self.x,self.weights_1))
            layer_2 = self.sigmoid(dot(layer_1,self.weights_2))
            layer_3 = self.sigmoid(dot(layer_2,self.weights_3))
            output = self.softmax(dot(layer_3,self.out_weights))
            output_final = np.argmax(output,axis=1)
#             print layer_1.shape
#             print layer_2.shape
#             print layer_3.shape
#             print output.shape
            
            
            # BACK PROPPOGATION 
            # Error
            outputError = self.y  - output_final # y - ypred
            print outputError
            error = (outputError**2).sum() / (2 * self.y.shape[0])
            errors.append(error)
                
            # delta = product of error and derivative of next layer
            #check if required to take the derivative of SOFTMAX also??????
            # only change in the derivative of output layer rest same
            print output.shape, outputError.shape
            delta  = outputError.dot(self.sigmoid_derivative(output))
            # print self.sigmoid_derivative(output_final).shape
            # print outputError.shape
            
            # Multiply with transpose of last layer
            print 'Layer3 shape: ', layer_3.shape
            print 'Delta shape:', delta.shape
            out_weights_adjustment = dot(layer_3.T,delta) 
            print self.out_weights.shape
            print out_weights_adjustment.shape
            #self.out_weights = np.argmax(self.out_weights,axis=1)
            self.out_weights +=  out_weights_adjustment #update weights
                
            # error = current_weight x delta in next layer
            delta = dot(delta,self.out_weights.T) * self.sigmoid_derivative(layer_3)
            weight_3_adjustment = dot(layer_2.T,delta)
            self.weights_3 += weight_3_adjustment
                
            delta = dot(delta,self.weights_3.T) * self.sigmoid_derivative(layer_2)
            weights_2_adjustment = dot(layer_1.T,delta)
            self.weights_2 += weights_2_adjustment
                
            delta = dot(delta,self.weights_2.T) * self.sigmoid_derivative(layer_1)
            weights_1_adjustment = dot(self.x.T,delta)
            self.weights_1 += weights_1_adjustment
        return errors                

In [173]:
import os
import numpy as np
import pandas as pd

if __name__ == '__main__':
    
    ds = pd.read_csv('../class_11/mnist/train.csv').values
    #x_train = ds[:5000,1:].reshape((-1,1,28,28)) / 255.0
    x_train = ds[:5000,1:]
    y_train = ds[:5000,0]

    #x_test = ds[5000:5002,1:].reshape(-1,1,28,28) / 255.0
    x_test = ds[5000:5010,1:]
    y_test = ds[5000:5010,0]
    
    
#     err = neural_network.train(200)
#     plt.plot(err)
#     plt.show()
    

In [178]:
neural_network = NeuralNet(x_train,y_train)
neural_network.train(3)

0
[-4 -6 -7 ...,  2  7  9]
(5000, 10) (5000,)
Layer3 shape:  (5000, 3)
Delta shape: (10,)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in exp


ValueError: shapes (3,5000) and (10,) not aligned: 5000 (dim 1) != 10 (dim 0)

In [164]:
predictions = neural_network.predict(x_test) 
print predictions.shape
output = np.argmax(predictions, axis=1)
print output
print y_test

(10, 784)
(784, 4)
(10, 10)
(10, 10)
[8 3 8 6 6 5 3 6 6 3]
[8 7 2 6 3 1 2 6 6 6]


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in exp


In [165]:
accuracy = (output == y_test).sum() * 100 / y_test.shape
print 'Accuracy = ' + str(accuracy) + '%'

Accuracy = [40]%


In [ ]:
# What do this shape signify?
# calculating the errors at end of training that are computed but what does this shape signify??

#accuracy = (outs == y_test).sum()*100.0 / y_test.shape[0] 


# Converting MNIST images to numpy arrays    
#     images_0 = []    
#     for root,dire,filenames in os.walk("mnistasjpg/trainingSet/0/."):        
#         for ix in range(1,100):
#             im = io.imread('mnistasjpg/trainingSet/0/' + str(filenames[ix]))            
#             images_0.append(im)
    
#     dataset = np.asarray(images_0)
#     print dataset.shape
#     x = dataset[0]
#     y = np.array([0,1])